# -*- coding: utf-8 -*-
# Created by Arunabh Athreya, January 24, 2024

# The script is created to perform the following functions:
    1. Membrane proximity trajectory of the COM of protein w.r.t the lipid bilayer
    2. contribution of individual residues in the protein-bilayer interaction
    3. specificity of interaction of residue vs type of lipid.
    4. magnitude of penetration into the bilayer at a global level.


In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Function to get the file path from the user
def get_file_path():
  while True:
    file_path = input("Enter the path to your file in Google Drive (e.g., /content/drive/MyDrive/your_file.txt): ")
    if os.path.exists(file_path):
      return file_path
    else:
      print("File not found. Please enter a valid path.")

# Get the file path from the user
file_path = get_file_path()

# Now you can access the file using the file_path variable
# For example, to read the file contents:
with open(file_path, 'r') as f:
  file_contents = f.read()
  # Process file_contents

In [ ]:
b=input("""enter the IDs of the lipids that were used in the simulation
and is seen in the PDB trajectory. For example, \n POPE,POPC,POPG,PMCL \n input:\n""")
name=input("Enter filename:\t")

In [ ]:
import numpy as np
import math
from scipy import stats

In [ ]:
"""
Block 1:
    This block helps in identifying the residues in the protein molecule and adds
    the coordinates of their Centre of Geometry (CoG) in a numpy array. Runs on
    only the first frame before reaching breakpoint.
"""
reslist=[]  #listing all residues in the protein.
prot_atoms=0
lipid_atoms=0
lipidcount=0
nframes=0   #frames of trajectory
content=open(str(name),'r').readlines()
lipids=b.split(",")
lipidlist=[]    #all residue numbers assigned to lipids
for line in content:
    if line.startswith("MODEL"):
        print(line)
    elif line.startswith("ATOM"):
        if line[21:22] == "A":
            prot_atoms=prot_atoms+1
            if line[22:26].strip() not in reslist:
                reslist.append(line[22:26].strip())
        for a in lipids:
            if a in line:
                lipid_atoms=lipid_atoms+1
                if line[22:26].strip() not in lipidlist:
                    lipidlist.append(line[22:26].strip())
    elif line.startswith("ENDMDL"):
        break
#print(lipidlist)
#print(len(lipidlist))
#print('\n'*3)
#print(reslist)
#print(len(reslist))
print("Number of protein atoms are: ", prot_atoms,'\n')
print("number of lipid atoms are: ",lipid_atoms,'\n')
for line in content:
    if line.startswith("MODEL"):
        nframes=nframes+1
        print("processed frame no.",nframes,'... \n')



    The arrays will have the following columns:
    column    class
    ---

    0         frame no.
    1         atom no.
    2         x-coord
    3         y-coord
    4         z-coord
    5         residue number
    6         atom-ID
    7         lipidtype
    8         in case of lipids: whether its a phosphorus atom (1) or not (0) in lipid_array.

In [ ]:
prot_array=np.ndarray(shape=(nframes,prot_atoms,7),dtype=object)
print("creating array in "+str(prot_array.ndim)+" dimensions for protein atoms.")
print(prot_array.shape)
#print(prot_array.dtype)
#print(prot_array)

lipid_array=np.ndarray(shape=(nframes,lipid_atoms,9),dtype=object)
print("creating array in "+str(lipid_array.ndim)+" dimensions for lipids.")
print(lipid_array.shape)
#print(lipid_array.dtype)
#print(lipid_array)


This block does the following:
1.   Adds all protein atom coordinates to prot_array
2.   Add all lipid atom coordinates to lipid_array
3.   Add an additional information of whether a certain line in lipid array belonged to phosphorus atom.

It also adds frame number for later purposes, when filtering the array for P
atoms will reduce the dimensions of the array.

In [ ]:
for line in content:
    if "MODEL" in line:
        rcount = 0
        lcount = 0
        model=int(line[6:14].strip())
        print(model)
    if line.startswith("ATOM"):
        if line[22:26].strip() in reslist:
            prot_array[model-1,rcount,0]=model
            prot_array[model-1,rcount,1]=line[5:11].strip()
            prot_array[model-1,rcount,2]=line[27:38].strip()
            prot_array[model-1,rcount,3]=line[38:46].strip()
            prot_array[model-1,rcount,4]=line[46:54].strip()
            prot_array[model-1,rcount,5]=line[22:26].strip()
            prot_array[model-1,rcount,6]=line[11:16].strip()
#            print(prot_array[model-1,rcount])
            rcount = rcount + 1
        if line[22:26].strip() in lipidlist:
            lipid_array[model-1,lcount,0]=model
            lipid_array[model-1,lcount,1]=line[5:11].strip()
            lipid_array[model-1,lcount,2]=line[27:38].strip()
            lipid_array[model-1,lcount,3]=line[38:46].strip()
            lipid_array[model-1,lcount,4]=line[46:54].strip()
            lipid_array[model-1,lcount,5]=line[22:26].strip()
            lipid_array[model-1,lcount,6]=line[11:16].strip()
            if 'P' in line[12:17].strip():  #adds a marker for a line of phosphorus
                lipid_array[model-1,lcount,8]=True
            for lipid in lipids:
                if lipid in line:
                    lipid_array[model-1,lcount,7]=lipid
            lcount = lcount + 1
print(prot_array)
print(lipid_array)

There are two functions of this block. The first is to determine the limits of the bilayer along the Z-axis. For this the path is:

1.   Find the consensus Z axis min and max
2.   Repeat for every frame

This helps in initiating further functions that calculate protein-lipid interacts only in frames where any atom of the protein is near the boundary of the lipid bilayer, saving computational time.

The second part of the code block generates a difference matrix for distance between every protein atom to every lipid atom in every frame.






In [ ]:
"""Iterate over all frames to find interacting atoms (between protein and lipid,
not lipid-lipid or intraprotein) to make an empty list to cumulatively store
diff_matrix (difference matrix) for each frame.
"""
# STEP-BY-STEP EXPLANATION:
# 1. Reading individual frames:
all_diff_matrices = []
lipid_count = {}
# all_diff_matrices: An empty list to store distance matrices (explained later) for each frame.
# lipid_count: An empty dictionary to store the count of interactions for each lipid type.
for frame in range(1,nframes+1):
    print(f"Processing frame {frame}...")
    diff_filter = [] # a subarray for protein atoms that come close to the bilayer
    # Create a mask for the current frame:
    bilayer_mask = lipid_array[:,:,0] == frame
    protein_mask = prot_array[:,:,0] == frame
    # Apply the mask to create the subarray
    lipid_subarray = lipid_array[bilayer_mask] #isolated all lipid data for one frame
    protein_subarray = prot_array[protein_mask] #isolated all protein data for one frame
    zcoord = lipid_subarray[:,4].astype(float) #get z-coordinates list for lipid array
    zmax = np.max(zcoord)
    zmin = np.min(zcoord)
    print(f"ZMax and Zmin are: '{zmax}','{zmin}'")
    # checking for atoms in the protein data whose either (Zcoord-zmin) or (Zcoord-zmax) <4:
    diffa=np.abs(protein_subarray[:,4].astype(float)-zmin) # Here Zcoordinate of a protein atom
    diffb=np.abs(protein_subarray[:,4].astype(float)-zmax) # is the 5th column in protein_subarray
    diff_filter=protein_subarray[np.logical_or(diffa < 4, diffb < 4)]
    print(f"There are {len(diff_filter)} protein atoms close to the bilayer in frame {frame}.")
    #print(diffb_filter)
    # Print the subarray for the current value
    #print(f"Frame '{frame}' protein:")
    #print(protein_subarray)
    print("-"*20)
    #print(f"Frame '{frame}' lipids:")
    #print(lipid_subarray)
    #print(f"The bilayer lies between '{zmin}' and '{zmax}'")
    if len(diff_filter) > 0:
        px = diff_filter[:, 2].astype(float) #Protein dataset's 3rd column, i.e., the X-coord.
        py = diff_filter[:, 3].astype(float) #Y-coord
        pz = diff_filter[:, 4].astype(float) #Z-coord
        lx = lipid_subarray[:, 2].astype(float) #lipid atom's X-coord
        ly = lipid_subarray[:, 3].astype(float) #lipid atom's Y-coord
        lz = lipid_subarray[:, 4].astype(float) #lipid atom's Z-coord
        diff_matrix = np.sqrt(np.power(px[:, np.newaxis] - lx, 2) +
                      np.power(py[:, np.newaxis] - ly, 2) +
                      np.power(pz[:, np.newaxis] - lz, 2))
        print(f"diff_matrix is: '{diff_matrix}'")
        # This will fetch the indices where diff_matrix values are <4Å
        row_indices, col_indices = np.where(diff_matrix < 4)
        print(f"row_indices are: '{row_indices}'")
        print(f"col_indices are: '{col_indices}'")
        # To extract the corresponding residue numbers from protein_array:
        prot_res_no = diff_filter[row_indices,5]
        lipi_res_no = lipid_subarray[col_indices,5]
        for row,col in zip(row_indices, col_indices):
            print(f"The interactions are: Residue {diff_filter[row,5]} and {lipid_subarray[col,7]} whose atoms {diff_filter[row,6]} and {lipid_subarray[col,6]} respectively are {diff_matrix[row, col]} Å apart.")
        print(prot_res_no,lipi_res_no)
        print(diff_matrix.shape) #should be (x,y), where x= number of protein atoms that
        # come close to bilayer, and y= total number of lipid atoms
        """
        To find which lipids interact the most:
          This will take the count of number of instances in the diff_matrix with
          values less than 4Å. Then it will ask to what kind of lipid do these instances
          belong to, by calling out lipid_subarray[col_indices,7] and appending it
          to a list, and then counting each lipidtype.
        """
        for col in col_indices:
            lipid_type = lipid_subarray[col, 7]
            if lipid_type in lipid_count:
                lipid_count[lipid_type] += 1
            else:
                lipid_count[lipid_type] = 1
    #print lipid counts here for each lipid type.
    print(f"cumulative lipid counts till frame {frame}:")
    for lipid_type, count in lipid_count.items():
        print(f"{lipid_type}: {count}")
    lipid_count.clear()
    all_diff_matrices.append(diff_matrix)
    print("=" * 20)  # Separator between frames